## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users\kkubi/Class Repo/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,65.98,37,92,6.35,overcast clouds
1,1,Port Lincoln,AU,-34.7333,135.8667,68.29,72,43,8.41,scattered clouds
2,2,Cootamundra,AU,-34.6500,148.0333,65.95,93,100,4.54,light rain
3,3,Nishihara,JP,26.1842,127.7558,67.66,55,40,13.80,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,44.26,70,75,11.50,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("On your trip, what is the coldest temperature you want it to be? "))
max_temp = float(input("On your trip, what is the hotest temperature you want it to be? " ))

On your trip, what is the coldest temperature you want it to be? 70
On your trip, what is the hotest temperature you want it to be? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Aden,YE,12.7794,45.0367,79.03,66,88,2.89,overcast clouds
7,7,Champerico,GT,14.3000,-91.9167,84.74,72,33,2.59,scattered clouds
19,19,Atuona,PF,-9.8000,-139.0333,77.88,77,27,12.93,scattered clouds
20,20,Souillac,MU,-20.5167,57.5167,72.36,88,40,4.61,scattered clouds
21,21,Vaini,TO,-21.2000,-175.2000,84.36,84,40,3.44,scattered clouds
23,23,Batemans Bay,AU,-35.7167,150.1833,78.19,64,100,5.70,light rain
24,24,Nacala,MZ,-14.5428,40.6728,79.00,84,36,10.29,scattered clouds
25,25,Bengkulu,ID,-3.8004,102.2655,75.49,83,100,2.66,overcast clouds
26,26,Kavieng,PG,-2.5744,150.7967,85.64,70,100,16.55,overcast clouds
27,27,Pitimbu,BR,-7.4706,-34.8086,79.39,76,31,10.65,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                237
City                   237
Country                237
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
preferred_cities_df.notnull().sum()

City_ID                237
City                   237
Country                237
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

### 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
###       I did this step in STEP #3 by placing dropna() outside the loc method's square brackets.

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Aden,YE,79.03,overcast clouds,12.7794,45.0367,
7,Champerico,GT,84.74,scattered clouds,14.3000,-91.9167,
19,Atuona,PF,77.88,scattered clouds,-9.8000,-139.0333,
20,Souillac,MU,72.36,scattered clouds,-20.5167,57.5167,
21,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,
23,Batemans Bay,AU,78.19,light rain,-35.7167,150.1833,
24,Nacala,MZ,79.00,scattered clouds,-14.5428,40.6728,
25,Bengkulu,ID,75.49,overcast clouds,-3.8004,102.2655,
26,Kavieng,PG,85.64,overcast clouds,-2.5744,150.7967,
27,Pitimbu,BR,79.39,scattered clouds,-7.4706,-34.8086,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
         
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Aden,YE,79.03,overcast clouds,12.7794,45.0367,Sama Emirate Hotel
7,Champerico,GT,84.74,scattered clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
19,Atuona,PF,77.88,scattered clouds,-9.8000,-139.0333,Villa Enata
20,Souillac,MU,72.36,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
21,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
655,Juba,SS,75.42,broken clouds,4.8517,31.5825,ROYAL PALACE HOTEL - JUBA
656,Cockburn Town,TC,80.56,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
658,Poya,NC,83.37,overcast clouds,-21.3500,165.1500,
663,Bossangoa,CF,71.29,overcast clouds,6.4926,17.4552,LA PERLE DE BOSTON


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Aden,YE,79.03,overcast clouds,12.7794,45.0367,Sama Emirate Hotel
7,Champerico,GT,84.74,scattered clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
19,Atuona,PF,77.88,scattered clouds,-9.8000,-139.0333,Villa Enata
20,Souillac,MU,72.36,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
21,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
655,Juba,SS,75.42,broken clouds,4.8517,31.5825,ROYAL PALACE HOTEL - JUBA
656,Cockburn Town,TC,80.56,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
658,Poya,NC,83.37,overcast clouds,-21.3500,165.1500,
663,Bossangoa,CF,71.29,overcast clouds,6.4926,17.4552,LA PERLE DE BOSTON


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))